<a href="https://colab.research.google.com/github/hyunjaec/MScA_Machine_Learning_Project/blob/main/medical_deepfakes_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
import os 
import seaborn as sns
try:
  import pydicom
except:
  ! pip install pydicom
  import pydicom
import scipy.ndimage

     |████████████████████████████████| 2.0 MB 4.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_dicom(path2scan_dir):
    dicom_folder = path2scan_dir
    dcms = os.listdir(dicom_folder)
    first_slice_data = pydicom.read_file(os.path.join(path2scan_dir,dcms[0]))
    first_slice = first_slice_data.pixel_array
    orientation = np.transpose(first_slice_data.ImageOrientationPatient) #zyx format
    spacing_xy = np.array(first_slice_data.PixelSpacing, dtype=float)
    spacing_z = np.float(first_slice_data.SliceThickness)
    spacing = np.array([spacing_z, spacing_xy[1], spacing_xy[0]]) #zyx format

    scan = np.zeros((len(dcms),first_slice.shape[0],first_slice.shape[1]))
    raw_slices=[]
    indexes = []
    for dcm in dcms:
        slice_data = pydicom.read_file(os.path.join(dicom_folder,dcm))
        slice_data.filename = dcm
        raw_slices.append(slice_data)
        indexes.append(float(slice_data.ImagePositionPatient[2]))
    indexes = np.array(indexes,dtype=float)

    raw_slices = [x for _, x in sorted(zip(indexes, raw_slices))]
    origin = np.array(raw_slices[0][0x00200032].value) #origin is assumed to be the image location of the first slice
    if origin is None:
        origin = np.zeros(3)
    else:
        origin = np.array([origin[2],origin[1],origin[0]]) #change from x,y,z to z,y,x

    for i, slice in enumerate(raw_slices):
        scan[i, :, :] = slice.pixel_array
    return scan, spacing, orientation, origin, raw_slices

In [ ]:
! ls 'drive/My Drive/data_ml'

'Experiment 2 - Open'
 labels_exp2.csv
'Response EXP2 - Reviewer 1_instances.csv'
'Response EXP2 - Reviewer 1_patients.csv'
'Response EXP2 - Reviewer 2_instances.csv'
'Response EXP2 - Reviewer 2_patients.csv'
'Response EXP2 - Reviewer 3_instances.csv'
'Response EXP2 - Reviewer 3_patients.csv'


In [ ]:
os.listdir('drive/My Drive/data_ml/Experiment 2 - Open')

['2220',
 '1563',
 '2575',
 '1876',
 '2031',
 '3341',
 '6031',
 '7507',
 '2592',
 '1251',
 '4709',
 '2960',
 '4635',
 '1796',
 '2366',
 '2590',
 '2199',
 '2495',
 '2703',
 '1610']

In [ ]:
from tqdm import tqdm 

scan_uuids = os.listdir('drive/My Drive/data_ml/Experiment 2 - Open')

scan = None
spacing = None
orientation = None
origin = None
raw_slices = []
for id in tqdm(scan_uuids):
  sc, sp, ori, org, rs = load_dicom('/content/drive/My Drive/data_ml/Experiment 2 - Open/'+id)
  if scan is None:
    scan = sc
  else:
    scan = np.concatenate((scan, sc))
  if spacing is None:
    spacing = sp
  else:
    spacing = np.concatenate((spacing, sp))
  if orientation is None:
    orientation = ori
  else:
    orientation = np.concatenate((orientation, ori))
  if origin is None:
    origin = org
  else:
    origin = np.concatenate((origin, org))
  raw_slices += rs

print('The CT scan has the dimensions of',scan.shape,'  (z,y,x)')

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
 35%|███▌      | 7/20 [01:31<02:07,  9.79s/it]

In [ ]:
from matplotlib import pyplot as plt

for slice_indx in range(50,100,10):
    plt.imshow(scan[slice_indx,:,:],cmap='bone',vmin=-1000,vmax=2000)
    plt.show()

In [ ]:
df = pd.read_csv('/content/drive/My Drive/data_ml/labels_exp2.csv')
locations = df.loc[df['uuid'].isin(scan_uuids)]
locations

In [ ]:
def cutCube(X, center, shape, padd=0): #center is a 3d coord (zyx)
    center = center.astype(int)
    hlz = np.round(shape[0] / 2)
    hly = np.round(shape[1] / 2)
    hlx = np.round(shape[2] / 2)

    #add padding if out of bounds
    if ((center - np.array([hlz,hly,hlx])) < 0).any() or (
        (center + np.array([hlz,hly,hlx]) + 1) > np.array(X.shape)).any():  # if cropping is out of bounds, add padding
        try:
          np.ones(np.array(X.shape) + shape * 2)
        except:
          return "No"
        Xn = np.ones(np.array(X.shape) + shape * 2) * padd
        Xn[shape[0]:(shape[0] + X.shape[0]), shape[1]:(shape[1] + X.shape[1]), shape[2]:(shape[2] + X.shape[2])] = X
        centern = center + shape
        cube = Xn[int(centern[0] - hlz):int(centern[0] - hlz + shape[0]),
               int(centern[1] - hly):int(centern[1] - hly + shape[1]),
               int(centern[2] - hlx):int(centern[2] - hlx + shape[2])]
        return np.copy(cube)
    else:
        cube = X[int(center[0] - hlz):int(center[0] - hlz + shape[0]), int(center[1] - hly):int(center[1] - hly + shape[1]),
               int(center[2] - hlx):int(center[2] - hlx + shape[2])]
        return np.copy(cube)


In [ ]:
cut_cubes = []
for i in range(len(locations)):
    location = locations.iloc[i]
    coord = np.array([location['slice'],location['y'],location['x']])
    cut_cubes.append(cutCube(scan,coord,(64,64,64)))

#disply the slices of each cut cube:
# for cube in cut_cubes:
#     plt.figure(num=None, figsize=(10, 10), dpi=200)
#     for i in range(64):
#         plt.subplot(8,8,i+1)
#         plt.axis('off')
#         plt.tight_layout()
#         try:
#           plt.imshow(cube[i,:,:],cmap='bone')
#         except:
#           print(i)
#     plt.show()

In [ ]:
import os
dir = "/content/drive/My Drive/Data/data_ml/Experiment 2 - Open/"
r=[]
for root, dirs, files in os.walk(dir):
  r.append(root)

In [ ]:
label_df1 = pd.read_csv('/content/drive/My Drive/data_ml/Response EXP2 - Reviewer 1_instances.csv')
label_df2 = pd.read_csv('/content/drive/My Drive/data_ml/Response EXP2 - Reviewer 2_instances.csv')
label_df3 = pd.read_csv('/content/drive/My Drive/data_ml/Response EXP2 - Reviewer 3_instances.csv')

preds = pd.concat([label_df1[['label','prediction','confidence']], label_df2[['label','prediction','confidence']], label_df3[['label','prediction','confidence']]], axis = 1)
preds.columns = ['label1','prediction1','confidence1','label2','prediction2','confidence2','label3','prediction3','confidence3']

def final_label(row):
  row.label1 = row.labe1.replace(False, -1)
  if row.label1 == row.label2 == row.label3:
    return row.label1
  else:
    return np.sign((row.label1*row.confidence1 + row.label1*row.confidence2 + row.label1*row.confidence3) / 3)

preds['label'] = preds.apply(lambda row: final_label(row), axis = 1)
label_df = pd.concat([label_df1.drop('label', axis = 1), preds['label']], axis = 1)
label_df.head()

In [ ]:
df_true = label_df.loc[label_df['label']==True]
df_false = label_df.loc[label_df['label']==False]
len(label_df)

In [ ]:
df_true.uuid[0]

In [ ]:
df_true = df_true.reset_index()
df_false = df_false.reset_index()

In [ ]:
from tqdm import tqdm

def image_cut_processing(df):
  cut_cubes = []
  for i in tqdm(range(0,len(df))):
    scan, spacing, orientation, origin, raw_slices = load_dicom('/content/drive/My Drive/data_ml/Experiment 2 - Open/'+str(df.uuid[i]))
    uuid_row = df.loc[df['uuid'] == df.uuid[i]]
    for j in range(len(uuid_row)):
      location = uuid_row.iloc[j]
      coord = np.array([location['slice'],location['y'],location['x']])
      if (cutCube(scan,coord,(64,64,64))) != "No":
        cut_cubes.append(cutCube(scan,coord,(64,64,64)))
  return cut_cubes
  

In [ ]:
true_cut_images = image_cut_processing(df_true)

In [ ]:
false_cut_images = image_cut_processing(df_false)

In [ ]:
list1 = [True for i in range(len(true_cut_images))]
list2 = [False for i in range(len(false_cut_images))]
y = list1+list2
y = np.array(y)

In [ ]:
y = pd.get_dummies(y)

In [ ]:
X = true_cut_images + false_cut_images

In [ ]:
len(X), len(y)

In [ ]:
y['label'] = (y[True] == 1).astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y['label'], test_size=.2)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
len(y_train), len(y_test)

In [ ]:
y.label.to_numpy()

In [ ]:
X_test[8].shape, y_train[8].shape

In [ ]:
X_train.shape

In [ ]:
nsamples, nx, ny, nz = X_train.shape
X_train_flattened = X_train.reshape((nsamples,nx*ny*nz))
X_test_flattened = X_test.reshape((X_test.shape[0],nx*ny*nz))

In [ ]:
X_train_flattened.shape

In [ ]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train_flattened, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

preds = svc.predict(X_test_flattened)

print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, preds))